# Exercise 4a 

Maximilian Jalea, Theresa Häberle

In [1]:
import pandas as pd                                     # for dealing with csv import
import numpy as np                                      # arrays and other matlab like manipulation
import os                                               # for joining paths and filenames sensibly

## 1. Loading and Cleaning the Data

In [2]:
filename=os.path.join('data','CrowdstormingDataJuly1st.csv') 
df = pd.read_csv(filename)
print(df.shape)

(146028, 28)


<font color=green> **COMMENTS:**
- data file loaded correctly
- decision to drop referee and related features is correct
- decision to drop birthday disagrees with solution provided
- they also dropped a number of relevant features such as position, club, number of victories, ties, etc. unfortunately    
</font>

We decide to drop the following features that we deem irrelevant for our analysis or redundant, e.g.: 
- The birthday is irrelevant since games from the whole career of each player is contained in the data and we don't know the dates of each game, and hence cannot compute the age of the time of the game.
- Almost all data describing the referees can be dropped
- The IAT and Exp values describing the racial bias in the referee countries can be dropped since taking those into the account would go beyond the scope of this exercise

In [3]:
df = df.drop(['player', 'club', 'birthday', 'position', 'victories', 'ties', 'defeats', 'photoID', 'refCountry', 'Alpha_3', 'meanIAT', 'nIAT', 'seIAT', 'meanExp', 'nExp', 'seExp'], axis=1)

Rows with missing skin color ratings (about 15% of all rows) will be deleted:

In [4]:
df = df.dropna(subset=['rater1', 'rater2'])
print(df.shape)

(124621, 12)


We only keep a single skin color rating value as the mean of the 2 ratings.

In [5]:
df.insert(10, 'rating', df[['rater1', 'rater2']].mean(axis=1))
df = df.drop(['rater1', 'rater2'], axis=1)

<font color=green> **COMMENTS:**
- remove nan data from skin color is done correctly
- Also correct to take the average of two ratings to represent skin tone.    
</font>

Referees with 21 or less interactions are probably not working for one of the major leagues, but were included in the dataset for some game in the early career of some player. Therefore, we deem them irrelevant for our analysis and remove them from the dataset. There are now about 76% of the original rows left.

After that, we can drop the refNum column.

In [6]:
allRefs = df.refNum.value_counts()
goodRefs = allRefs[allRefs>21]
df = df[~df['refNum'].isin(goodRefs)]

df = df.drop(['refNum'], axis=1)

In [7]:
df = df.groupby(['playerShort', 'height', 'weight', 'rating', 'leagueCountry']).sum()
df = df.reset_index()

,playerShort,height,weight,rating,leagueCountry,games,goals,yellowCards,yellowReds,redCards
0,aaron-hughes,182.0,71.0,0.125,England,575,8,14,0,0
1,aaron-hunt,183.0,73.0,0.125,Germany,326,60,41,0,1
2,aaron-lennon,165.0,63.0,0.250,England,353,30,9,0,0
3,aaron-ramsey,178.0,76.0,0.000,England,218,30,26,0,0
4,abdelhamid-el-kaoutari,180.0,73.0,0.250,France,107,1,7,4,2
...,...,...,...,...,...,...,...,...,...,...
1559,zoltan-gera,181.0,76.0,0.250,England,345,64,41,1,1
1560,zoltan-stieber,175.0,67.0,0.000,Germany,133,24,12,0,0
1561,zoumana-camara,182.0,76.0,0.875,France,354,7,42,2,5
1562,zubikarai,185.0,84.0,0.000,Spain,35,0,2,0,1


As proposed in the exercise, the categorial feature 'leagueCountry', which describes the league of each player in the 2012/13 season, is one-hot-encoded.

In [8]:
df = pd.concat([df.drop('leagueCountry', axis=1), pd.get_dummies(df['leagueCountry'], prefix='league_')], axis=1)

In [9]:
df

,playerShort,height,weight,rating,games,goals,yellowCards,yellowReds,redCards,league__England,league__France,league__Germany,league__Spain
0,aaron-hughes,182.0,71.0,0.125,575,8,14,0,0,1,0,0,0
1,aaron-hunt,183.0,73.0,0.125,326,60,41,0,1,0,0,1,0
2,aaron-lennon,165.0,63.0,0.250,353,30,9,0,0,1,0,0,0
3,aaron-ramsey,178.0,76.0,0.000,218,30,26,0,0,1,0,0,0
4,abdelhamid-el-kaoutari,180.0,73.0,0.250,107,1,7,4,2,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1559,zoltan-gera,181.0,76.0,0.250,345,64,41,1,1,1,0,0,0
1560,zoltan-stieber,175.0,67.0,0.000,133,24,12,0,0,0,0,1,0
1561,zoumana-camara,182.0,76.0,0.875,354,7,42,2,5,0,1,0,0
1562,zubikarai,185.0,84.0,0.000,35,0,2,0,1,0,0,0,1


<font color=green> **COMMENTS:**
- onehot coding for categorical feature league country is done correctly    
</font>

Now we still have to calculate the correct responses $ Y_i = N_{i, red} / N_i $ for each player.
We decide to treat yellowReds just like redCards.

In [10]:
df.insert(13, 'target', (df['redCards'] + df['yellowReds']) / df['games'])

df

For linear regression as treated in the lecture, we need to center the data, i. e. subtract the mean of each column from every value in the column, such that each column has mean zero.

In [17]:
df_centered = df
df_centered[[height]] = df_centered[height] - df_centered[height].mean

df_centered



NameError: name 'height' is not defined

<font color=green> **COMMENTS:**
- syntax error in selecting a column from dataframe, should be df['height']    
</font>

## 2. Model Creation

### Linear Regression


### Regression forest

<font color=green> **COMMENTS:**
- part 2 model selection and subsequent analysis are not completed
- you could at least implemented the DecisionTree and DecisionForest since they should be exactly the same as the solution already provided last week...
</font>